In [14]:
import sys
sys.path.extend(['/Users/zsolt/Colloid_git/TractionRheoscopy'])
from data_analysis import static as da
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from importlib import reload
import numpy as np

In [87]:
hdf_stem = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/'
sed_fName = 'tfrGel10212018A_shearRun10292018f_sed_stitched.h5'
gel_fName = 'tfrGel10212018A_shearRun10292018f_gel_stitched.h5'

figPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots'

In [3]:
gelPos = da.loadData2Mem(hdf_stem + gel_fName)
sedPos = da.loadData2Mem(hdf_stem + sed_fName)

In [4]:
gelFitPlane_dict = da.fitTopSurface(gelPos)
da.distFromPlane(gelPos,'z (um, below gel)',gelFitPlane_dict)
gelStrain_traj = da.gelStrain(gelPos,160)

/Users/zsolt/Colloid_git/TractionRheoscopy/data_analysis/static.py:124: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  fit, residual, rank, s = np.linalg.lstsq(A, z)


# Spatial selection of local strain in sediment

In [5]:
sed_strainTraj_dt3 = pd.read_hdf(hdf_stem + '/{}'.format('tfrGel10212018A_shearRun10292018f_sed_strainTraj_dt3.h5'))
                                                        

In [81]:
sed_strainTraj_refConfig = pd.read_hdf(hdf_stem + '/{}'.format('tfrGel10212018A_shearRun10292018f_sed_strainTrajZeroRef.h5'))

In [6]:
da.distFromPlane(sedPos,'height above gel (um)', gelFitPlane_dict)

# Von Mises strain 

In [7]:
def vonMises(e11,e12,e13,e22,e23,e33): return np.sqrt(((e11-e22)**2 + (e22-e33)**2 + (e33-e11)**2 +6*(e12**2 + e23**2 + e13**2))/2)

In [122]:
#sed_strainTraj_dt3.unstack().reindex(idx).loc[:,buf.columns.get_level_values(1) =='eyz'].droplevel(1,axis=1)
idx = sed_strainTraj_refConfig[sed_strainTraj_refConfig.loc[(slice(None),'nnb count'),:] >= 9].dropna().unstack().index
buf2 = sed_strainTraj_refConfig.unstack().reindex(idx)

In [82]:
exx = buf2.loc[:,buf2.columns.get_level_values(1) == 'exx'].droplevel(1,axis=1).to_numpy()
exy = buf2.loc[:,buf2.columns.get_level_values(1) == 'exy'].droplevel(1,axis=1).to_numpy()
exz = buf2.loc[:,buf2.columns.get_level_values(1) == 'exz'].droplevel(1,axis=1).to_numpy()
eyy = buf2.loc[:,buf2.columns.get_level_values(1) == 'eyy'].droplevel(1,axis=1).to_numpy()
eyz = buf2.loc[:,buf2.columns.get_level_values(1) == 'eyz'].droplevel(1,axis=1).to_numpy()
ezz = buf2.loc[:,buf2.columns.get_level_values(1) == 'ezz'].droplevel(1,axis=1).to_numpy()

In [83]:
vonMises_np = vonMises(exx,exy,exz,eyy,eyz,ezz)
vonMises_df = pd.DataFrame(vonMises_np,index=idx,columns=buf2.loc[:,buf2.columns.get_level_values(1) == 'exx'].droplevel(1,axis=1).columns)

In [156]:
#%%capture 
pos0 = sedPos.xs(0,level='frame').loc[idx]
vonMises_df['xbin'] = pd.cut(pos0['x (um, imageStack)'], 40)
vonMises_df['ybin'] = pd.cut(pos0['y (um, imageStack)'], 40)
#vonMises_df['zbin'] = pd.cut(pos0['z (um, imageStack)'], 10)

tmp = vonMises_df.groupby(['xbin','ybin']).mean().apply(lambda x: x*100)
#tmp = vonMises_df.groupby(['xbin','ybin','zbin']).mean().apply(lambda x: x*100)
#tmp = vonMises_df.groupby(['xbin','ybin']).mean().apply(lambda x: x*100)
#vonMises_df
#tmp

In [157]:
#tmp = vonMises_df.groupby(['xbin','ybin','zbin'])
#tmp.loc[(slice(None),slice(None),pd.Interval(left=26.461,right=32.444)),:].unstack().droplevel(1,axis=1)

In [161]:
%%capture
plt.close('all')
for t in vonMises_df.columns[0:-3]:
    sns.heatmap(tmp[t].unstack(),cmap='YlGn',vmin=0,vmax=8);
    t_fName = 't{}'.format(t.replace('(','').replace(')','').replace(',','_'))
    plt.title('Deviatoric strain (J2) \n averaged over ~5um square boxes');
    plt.savefig(figPath+'/{}/{}'.format('sedStrain/localStrain','vonMises_localStrain_refConfig_{}.png'.format(t_fName)),dpi=300,bbox_inches='tight');
    plt.close();

# Von Mises yield surface

## Plan

+ get eignenvectors of strain tensor
- see if there is an approximate 1D (simple shear) or 2D (plane strain) subspace
- for each strain tensor, project eigenvector onto basis to generate point on von mises yield surface. 
- color point by d2min value or maybe some coarse graining... idea is that isocontours of d2min will define yield surface
- do this for every time point and animate
- next step would be to plot the tensor field and look for spatial correlations
    - what do deviatoric strain isocontours on eshebly inclusions look like?
    - can we do correlations, Aidan style, on deviatoric strain? 




In [97]:
#strain_micro = sed_strainTraj_dt3.dropna().head(11*100000)
strain_keys = ['exx','exy','exz','eyy','eyz','ezz']
strain_micro = sed_strainTraj_refConfig.dropna().loc[(slice(None),strain_keys),:].iloc[:,[25]].unstack()

In [98]:
d2_min_micro = sed_strainTraj_refConfig.dropna().loc[(slice(None),'D2_min'),:].iloc[:,[25]].unstack()

In [99]:
d2_min_micro.columns = d2_min_micro.columns.droplevel(0)
d2_min_micro = d2_min_micro.squeeze()

In [93]:
def strainDiag(strain_1d):
    exx,exy,exz,eyy,eyz,ezz = strain_1d
    e = np.array([[exx, exy, exz],
                  [exy, eyy, eyz],
                  [exz, eyz, ezz]])
    eigen_val, eigen_vec = np.linalg.eig(e)
    # now sort by eigen value
    idx = eigen_val.argsort()[::-1]
    # return sorted val and vec
    tmp = eigen_val[idx],eigen_vec[:,idx]
    return np.array([np.concatenate((tmp[0][n],tmp[1][n]),axis=None) for n in range(3)]).flatten()

In [105]:
# plot eigenvalues
#eigen_micro = np.array([strainDiag(elt)[0] for elt in 
#                        strain_micro.loc[(slice(None),strain_keys),:].iloc[:,[0]].to_numpy().reshape(100000,6)]);

eigen_micro = [strainDiag(elt) for elt in strain_micro.to_numpy()];

In [106]:
# This is a key step...
#.  -> form a multi-index from the particle index in strain_micro and all combinations with u,v,w eigen vector label
#.  -> reshape the array to be stacked 4x3 output for single particle as opposed to single line of 12 values lambda_u, ux,uy,...,lambda_w, wx,wy,wz
m_idx = pd.MultiIndex.from_product([strain_micro.index,['u','v','w']],names=['particle','eigen vector'])
eigen_df = pd.DataFrame(np.array(eigen_micro).reshape(-1,4),index=m_idx,columns=['lambda','x','y','z'])

In [107]:
eigen_df

lambda         x         y         z
particle eigen vector                                        
50       u             0.038190  0.742495  0.665326  0.077732
         v             0.026383  0.155132 -0.057906 -0.986195
         w            -0.011174  0.651640 -0.744304  0.146209
56       u             0.009465 -0.188395 -0.735238 -0.651101
         v             0.003759 -0.760634  0.528613 -0.376834
...                         ...       ...       ...       ...
966612   v            -0.001263 -0.749824 -0.619987 -0.231039
         w            -0.003239 -0.483780  0.751960 -0.447786
966614   u             0.024572 -0.111132 -0.581544  0.805888
         v            -0.013769  0.776039 -0.557346 -0.295176
         w            -0.017480  0.620817  0.592597  0.513240

[2719236 rows x 4 columns]

In [103]:
#np.array(eigen_micro).shape
lambda_u = eigen_df.xs('u',level='eigen vector')['lambda']
lambda_v = eigen_df.xs('v',level='eigen vector')['lambda']
lambda_w = eigen_df.xs('w',level='eigen vector')['lambda']
sns.distplot(lambda_u[lambda_u <1] - lambda_w[lambda_w >-1],kde=False)

#sns.distplot(eigen_micro[:,-1],kde=False)
#sns.distplot(eigen_micro[:,0] - eigen_micro[:,-1],kde=False)

<AxesSubplot:xlabel='lambda'>

In [104]:
sns.distplot(lambda_u[(lambda_u <0.06)],kde=False,label='lambda_u')
sns.distplot(lambda_v[(lambda_v <0.02) & (lambda_v > -0.02)],kde=False,label='lambda_v')
sns.distplot(lambda_w[(lambda_w >-0.06)],kde=False,label='lambda_w')
plt.legend()

In [109]:
%matplotlib widget
idx = (lambda_u <0.05) & (lambda_w >-0.05)
sns.jointplot(y=lambda_u[idx],x=lambda_w[idx] ,kind='hex')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
idx = (lambda_u <0.04) & (lambda_w >-0.04)
idx = (lambda_u <0.05) & (lambda_w >-0.05)
sns.jointplot(y=lambda_u[idx],x=lambda_w[idx] ,kind='hex')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
sns.jointplot(y=lambda_u[idx],x=-1*lambda_u[idx] ,kind='hex')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
idx = (lambda_u <0.04) & (lambda_w >-0.04) & (d2_min_micro <1.2)
sns.jointplot(x=d2_min_micro[idx], y=(lambda_u[idx] - lambda_w[idx]),kind='hex')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# von Mises volumetric visualization

In [162]:
# assign x,y, and z boxes to each particle 
pos0 = sedPos.xs(0,level='frame').loc[idx]
vonMises_df['xbin'] = pd.cut(pos0['x (um, imageStack)'], 40)
vonMises_df['ybin'] = pd.cut(pos0['y (um, imageStack)'], 40)
vonMises_df['zbin'] = pd.cut(pos0['z (um, imageStack)'], 10)

In [181]:
# average over boxes and group slow to fast to get z,y,x indexing 
tmp = vonMises_df.groupby(['zbin', 'ybin','xbin']).mean().apply(lambda x: x*100)

In [196]:
from particleLocating.flatField import array2tif

In [219]:
def rescaleArray(imgArray,minVal, maxVal, dtypeOut='uint16'):
    if dtypeOut != 'uint16': raise KeyError('not implemented')
    else:
        m = 2**16/(maxVal - minVal)
        b = 2**16 - m*maxVal
        mArray = np.full(imgArray.shape, m)
        bArray = np.full(imgArray.shape, b)
        rescaled = np.array(np.multiply(mArray,imgArray) + bArray)
        clipped = np.clip(rescaled,0,2**16-1)
        return clipped.astype('uint16')

In [220]:
plt.close('all')
figPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots'
out = np.zeros((89,10,40,40),dtype='uint16')
n = 0
for t in vonMises_df.columns[0:-3]:
    minVal,maxVal = (0,12)
    tmp_t_np = tmp[t].to_numpy().reshape(10,40,40)
    tmp_t_16bit = rescaleArray(tmp_t_np,minVal,maxVal)
    out[n,:,:,:] = tmp_t_16bit
    n += 1

In [222]:
fName = figPath +'/{}/{}'.format('vonMises_coarseGrained','vonMises_coarseGrained_localStrain_refConfig.tif'.format(t_fName))
#array2tif(tmp_t_16bit,fName,metaData='xbins:40, ybins:40, zBins=10, minVal:0, maxVal:53, description: coarse grained vonMises strain')

In [224]:
import skimage as ski
metaData_str = 'xbins:40, ybins:40, zBins=10, minVal:0, maxVal:12, description: coarse grained vonMises strain'
with ski.external.tifffile.TiffWriter(fName, imagej=True) as tif:
    tif.save(out,description = metaData_str)

/Users/zsolt/.conda/envs/TractionRheoscopy/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:648: UserWarning: not writing description to ImageJ file
  warnings.warn("not writing description to ImageJ file")


In [210]:
out.dtype

dtype('float64')

In [215]:
max(tmp_t_np.flatten())

52.505416166282124

## Ovito output of vonMises mag and normal plane for a single time

In [225]:
# for a single time step, create a dataFrame of position, vonMises magnitude, and middle eigen vector components
pos0

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x_std,y_std,z_std,height above gel (um)
particle,,,,,,,
56,45.569391,27.481420,27.466238,0.092833,0.095235,0.098554,3.226896
66,19.175736,3.288491,28.720584,0.218239,0.214715,0.246054,4.758069
69,9.055759,4.064479,27.535149,0.130883,0.132533,0.111610,3.514087
85,5.379716,7.916556,27.522730,0.177642,0.160699,0.146733,3.421909
99,12.489961,10.384146,27.502647,0.120851,0.118578,0.112423,3.393600
...,...,...,...,...,...,...,...
966599,221.162941,222.627548,85.054128,0.607085,0.597758,0.470942,58.412879
966606,215.785492,217.283723,85.509140,0.331072,0.332516,0.270769,58.931087
966611,192.962719,212.016256,69.874427,0.089947,0.092212,0.089670,43.279308


In [226]:
vonMises_df

,"(0,1)","(0,2)","(0,3)","(0,4)","(0,5)","(0,6)","(0,7)","(0,8)","(0,9)","(0,10)",...,"(0,83)","(0,84)","(0,85)","(0,86)","(0,87)","(0,88)","(0,89)",xbin,ybin,zbin
particle,,,,,,,,,,,,,,,,,,,,,
56,0.024983,0.026404,0.016012,0.030565,0.014082,0.027627,0.052025,0.029337,0.038629,0.031433,...,0.027572,0.024685,0.044921,0.018638,0.027881,0.029264,0.025110,"(41.302, 46.933]","(25.094, 30.883]","(26.461, 32.444]"
66,0.039894,0.036507,0.024771,0.034223,0.035546,0.087250,0.045776,0.059114,0.028867,0.040236,...,0.097919,0.026186,0.027486,0.198163,0.129708,0.095534,0.083967,"(18.777, 24.408]","(1.708, 7.728]","(26.461, 32.444]"
69,0.049350,0.048124,0.047311,0.045653,0.029484,0.027808,0.045086,0.035589,0.062102,0.035258,...,0.069795,0.037864,0.034677,0.066682,0.062313,0.039446,0.050033,"(7.515, 13.146]","(1.708, 7.728]","(26.461, 32.444]"
85,0.066554,0.058360,0.078276,0.052163,0.051854,0.066433,0.050816,0.058873,0.098564,0.023418,...,0.071993,0.060835,0.033195,0.061639,0.067515,0.045868,0.014118,"(1.658, 7.515]","(7.728, 13.517]","(26.461, 32.444]"
99,0.040952,0.028810,0.042829,0.021401,0.060931,0.039863,0.052505,0.037535,0.046717,0.078913,...,0.064416,0.036103,0.013655,0.043845,0.033929,0.065111,0.063758,"(7.515, 13.146]","(7.728, 13.517]","(26.461, 32.444]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966599,0.021313,0.034359,0.088363,0.025686,0.054553,0.066912,0.039543,0.057342,0.060940,0.075948,...,0.229099,0.227819,0.231545,0.219750,0.240626,0.226899,0.264603,"(215.868, 221.499]","(221.911, 227.7]","(79.836, 85.76]"
966606,0.029386,0.036987,0.044086,0.043269,0.020067,0.036337,0.056639,0.019151,0.035972,0.072636,...,0.103752,0.130978,0.113214,0.112008,0.112086,0.114017,0.140489,"(210.236, 215.868]","(216.122, 221.911]","(79.836, 85.76]"
966611,0.017839,0.016011,0.018039,0.025150,0.034916,0.038303,0.031547,0.022862,0.039681,0.030708,...,0.030632,0.035705,0.033190,0.034369,0.038959,0.032229,0.040207,"(187.712, 193.343]","(210.334, 216.122]","(67.988, 73.912]"


In [227]:
strain_micro

(0,26)                                                  
values         exx       exy       exz       eyy       eyz       ezz
particle                                                            
50        0.032665  0.004239  0.005286 -0.009860  0.006609  0.030594
56       -0.009897 -0.007203 -0.014690  0.002418 -0.010812 -0.008228
57       -0.001931  0.001100 -0.012222  0.012288  0.012128  0.009037
61        0.009285 -0.017842 -0.027286 -0.002437 -0.003265  0.010426
65        0.023276  0.007341  0.007565 -0.014037 -0.045497 -0.071248
...            ...       ...       ...       ...       ...       ...
966608    0.013684 -0.000849 -0.014816  0.023999 -0.019624  0.026618
966609   -0.011820  0.005659 -0.002194 -0.008936  0.008721  0.018419
966611    0.010670  0.007753  0.022680 -0.002405 -0.006300 -0.008135
966612    0.003577 -0.010885 -0.007533  0.016059  0.011039  0.005596
966614   -0.015706 -0.002424 -0.004180  0.008998  0.019034  0.000030

[906412 rows x 6 columns]

In [230]:
pd.join(pos0,eigen_df.xs('v',level='eigen vector'))

AttributeError: module 'pandas' has no attribute 'join'

In [239]:
coord = ['x (um, imageStack)', 'y (um, imageStack)','z (um, imageStack)']
vonMises_vecField = pos0[coord].join(eigen_df.xs('v',level='eigen vector')[['x','y','z']]).join(vonMises_df['(0,26)']).dropna()

In [240]:
vonMises_vecField

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x,y,z,"(0,26)"
particle,,,,,,,
56,45.569391,27.481420,27.466238,-0.760634,0.528613,-0.376834,0.035884
66,19.175736,3.288491,28.720584,-0.789974,0.283681,0.543568,0.022842
69,9.055759,4.064479,27.535149,0.290745,0.107851,-0.950702,0.056078
85,5.379716,7.916556,27.522730,-0.018520,0.988804,0.148068,0.061264
99,12.489961,10.384146,27.502647,-0.244841,-0.838456,0.486872,0.041772
...,...,...,...,...,...,...,...
966599,221.162941,222.627548,85.054128,0.720090,0.639449,-0.269397,0.084156
966606,215.785492,217.283723,85.509140,-0.344536,0.295117,0.891179,0.056394
966611,192.962719,212.016256,69.874427,0.109698,-0.925037,0.363694,0.046057


In [245]:
with open(figPath + '/tmp.xyz','w') as f:
    f.write(str(vonMises_vecField.shape[0])+'\n\n')
    vonMises_vecField.to_csv(f,sep=' ',header=False, index=False)

In [244]:
vonMises_vecField.shape[0]

890920

In [246]:
figPath

'/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots'

In [248]:
strainDiag([0,0,0,0,0.1,0])

array([ 0.1       ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.70710678,  0.        ,  0.70710678, -0.1       ,  0.70710678,
        0.        , -0.70710678])

## Ovito output of position and von Mises strain at series of time points (sequence of xyz files)

In [249]:
# select only particles that have more than 9nnb for the entire trajectory
idx = sed_strainTraj_refConfig[sed_strainTraj_refConfig.loc[(slice(None),'nnb count'),:] >= 9].dropna().unstack().index

In [255]:
vonMises_df.columns[0:-3]
sedPos.xs(89,level='frame').loc[idx]

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)",x_std,y_std,z_std,height above gel (um)
particle,,,,,,,
56,45.803781,27.347034,27.173402,0.083586,0.085046,0.086040,3.208465
66,19.448406,3.164594,28.484108,0.187832,0.190978,0.221804,4.780282
69,9.308049,3.984946,27.156524,0.133882,0.128912,0.110028,3.382359
85,5.606421,7.850624,27.147248,0.221733,0.218456,0.203487,3.287245
99,12.737096,10.340129,27.165349,0.114668,0.111147,0.111660,3.303079
...,...,...,...,...,...,...,...
966599,221.946982,222.083822,84.424353,0.512242,0.501275,0.433975,58.155642
966606,216.003449,217.251090,85.182190,0.789782,0.788171,0.660867,58.961891
966611,193.165376,211.876790,69.524291,0.101458,0.102991,0.107771,43.267409


In [264]:
# now loop over time and create tmp data frame of positions and von Mises strain values
coord = ['x (um, imageStack)', 'y (um, imageStack)','z (um, imageStack)']
f_stem = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots/vonMises_strain_traj'

n = 1
for t in vonMises_df.columns[0:-3]:
    out_df = sedPos.xs(n,level='frame').loc[idx][coord].join(vonMises_df[t]).dropna().sort_index(inplace=False)
    
    with open(f_stem + '/vonMises_refConfig_{:02d}.xyz'.format(n),'w') as f:
        f.write(str(vonMises_vecField.shape[0])+'\n\n')
        out_df.to_csv(f,sep=' ',header=False, index=False)
    
    n += 1

In [257]:
out_df.sort_index(inplace=False)

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)","(0,89)"
particle,,,,
56,45.609338,27.508988,27.487270,0.025110
66,19.213985,3.283517,28.722684,0.083967
69,9.037422,4.055317,27.501459,0.050033
85,5.398072,7.948800,27.568555,0.014118
99,12.488210,10.424394,27.486755,0.063758
...,...,...,...,...
966599,221.326708,222.672703,85.038769,0.264603
966606,215.868926,217.336446,85.439468,0.140489
966611,193.034000,212.095896,69.845486,0.040207


In [261]:
'ad_{:02d}'.format(30)

'ad_30'

In [265]:
out_df.sort_index(inplace=False)

,"x (um, imageStack)","y (um, imageStack)","z (um, imageStack)","(0,89)"
particle,,,,
56,45.803781,27.347034,27.173402,0.025110
66,19.448406,3.164594,28.484108,0.083967
69,9.308049,3.984946,27.156524,0.050033
85,5.606421,7.850624,27.147248,0.014118
99,12.737096,10.340129,27.165349,0.063758
...,...,...,...,...
966599,221.946982,222.083822,84.424353,0.264603
966606,216.003449,217.251090,85.182190,0.140489
966611,193.165376,211.876790,69.524291,0.040207
